# Tests for DataMux

In [ ]:
import logging
logging.basicConfig(format="%(asctime)-15s %(message)s", level=logging.INFO)
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from serving import WebsocketClient

server_host = "localhost"
server_port = 3300

client = WebsocketClient(server_host, server_port, backend="avro")
await client.connect()

In [ ]:
import asyncio

collections = None
collection_streams = None
live_streams = None


async def log_messages():
    global collections
    global collection_streams
    global live_streams

    while True:
        payload = await client.recvmsg()
        if payload is not None:
            topic, content = payload
            if topic == client.protocol.LIST_COLLECTIONS:
                collections = content["collections"]
            if topic == client.protocol.LIST_COLLECTION_STREAMS:
                collection_streams = content["streams"]
            if topic == client.protocol.LIST_LIVE_STREAMS:
                live_streams = content["streams"]


logging_task = asyncio.create_task(log_messages())

In [ ]:
await client.sendmsg(*client.datamux.list_collections())
await asyncio.sleep(2)
assert collections

In [ ]:
await client.sendmsg(*client.datamux.list_collection_streams("adhd_sin"))
await asyncio.sleep(2)
assert collection_streams

In [ ]:
collection_name = "adhd_sin"
stream_name = "Gaze"
attrs = dict(
    {
        "diagnosis": "Non-ADHD",
        "gender": "female",
        "noise": "0",
        "participant": "12",
        "question": "10",
        "collection": "adhd_sin",
        "id": "gaze",
    }
)

In [ ]:
await client.sendmsg(
    *client.datamux.replay_collection_stream(collection_name, stream_name, attrs)
)
await asyncio.sleep(10)

In [ ]:
await client.sendmsg(
    *client.datamux.restream_collection_stream(collection_name, stream_name, attrs)
)
await asyncio.sleep(2)

In [ ]:
await client.sendmsg(*client.datamux.list_live_streams())
await asyncio.sleep(2)
assert live_streams

In [ ]:
assert live_streams
live_stream = live_streams[0]

ls_name = live_stream["name"]
ls_attrs = live_stream["attrs"]
await client.sendmsg(*client.datamux.relay_live_streams(ls_name, ls_attrs))